<a href="https://colab.research.google.com/github/ankritRisal/Finetuning_LLM/blob/main/Train_finetunned_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Play with data



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd

In [314]:
df = pd.read_csv("/content/drive/MyDrive/X3s4c5/FinetunningLLMmodels/book_description.csv")

In [315]:
df.head(5)

,Unnamed: 0.1,Unnamed: 0,Title,Category,Price,Avilability,Book_Description,Stars
0,0,0,A Light in the Attic,Poetry,51.77,22,It's hard to imagine a world without A Light i...,3
1,1,1,Tipping the Velvet,Historical Fiction,53.74,20,"""Erotic and absorbing...Written with starling ...",1
2,2,2,Soumission,Fiction,50.10,20,"Dans une France assez proche de la nÃ´tre, un ...",1
3,3,3,Sharp Objects,Mystery,47.82,20,"WICKED above her hipbone, GIRL across her hear...",4
4,4,4,Sapiens: A Brief History of Humankind,History,54.23,20,From a renowned historian comes a groundbreaki...,5


In [316]:
df.drop(["Unnamed: 0", "Unnamed: 0.1", "Price", "Avilability", "Stars", ], axis = 1, inplace = True)

In [317]:
df.head(5)

,Title,Category,Book_Description
0,A Light in the Attic,Poetry,It's hard to imagine a world without A Light i...
1,Tipping the Velvet,Historical Fiction,"""Erotic and absorbing...Written with starling ..."
2,Soumission,Fiction,"Dans une France assez proche de la nÃ´tre, un ..."
3,Sharp Objects,Mystery,"WICKED above her hipbone, GIRL across her hear..."
4,Sapiens: A Brief History of Humankind,History,From a renowned historian comes a groundbreaki...


In [318]:
random_seed = 32
df = df.sample(frac=1, random_state = random_seed).reset_index(drop=True).reset_index()
train_size = 0.8
train_len = int(train_size * len(df))

In [319]:
df_train= df[:train_len]
df_test = df[train_len:]

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

from torch.utils.data import Dataset, DataLoader

from google.colab import userdata
from huggingface_hub import login

# Get the token from Colab secrets
hf_token = userdata.get('HF_TOKEN')

login(token=hf_token)


In [10]:
model_id = "meta-llama/Llama-3.2-3B-Instruct"

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side ="left")
# tokenizer.pad_token = tokenizer.eos_token # padding tokens to make of same shape

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             dtype = torch.bfloat16,
                                             device_map = device)

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [371]:
valid_category = list(df["Category"].unique())
system_prompt =  \
  {
    "role" : "system",
    "content" : f""" You are an AI system that reads an Title and Book Description and classifies category of the book applied, you must
    choose from the following classes:
    {"\n or ". join(["Labeled Category:" + x for x in list(valid_category)])}.
    Ensure Output is from above list only"""
    }

post_messages = \
  {
    "role" : "assistant",
    "content" : "Labeled Category"
    }

# user_messages = {
#       "role" : "user",
#       "content" : """ Title :A Light in the Attic
#                     Book Description: It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place to rock?And who put you up there,And your cradle, too?Baby,
#                     I think someone down here'sGot it in for you. Shel, you never sounded so good. ...more"""
#     }
prompt = [
    [system_prompt, post_messages]
  ]

In [321]:
def extract_labels_from_output(decode_batch, tokenizer):
  # labels = []
  # for d in decode_batch:
  #   if "Labeled Category:" in d:
  #       label = d.split("Labeled Category:")[1].split("<|eot_id|>")[0].strip()
  #       return labels
  #   else:
  #       label = None
  #   labels.append(label)
  labels  = [d.split("<|start_header_id|>assistant<|end_header_id|>\n\nLabeled Category:")[1].split("<|eot_id|>")[0].strip() for d in decode_batch]
  return labels

In [322]:
def generate_outputs(prompts, model, tokenizer):
  tokenizer.pad_token = tokenizer.eos_token
  tokenized = tokenizer(prompts, padding= True, return_tensors ="pt", add_special_tokens= False).to(device)

  # tokenized = tokenizer.apply_chat_template(prompts, padding= False, return_tensors ="pt", add_special_tokens= False).to(device)
  output_batch = model.generate(input_ids = tokenized["input_ids"], attention_mask =tokenized["attention_mask"], max_new_tokens = 20, do_sample= False, temperature = 0, top_p =1)
  # output_batch = model.generate(tokenized, max_new_tokens = 20)
  decode_batch = tokenizer.batch_decode(output_batch, skip_special_tokens= True)
  prediction = extract_labels_from_output(decode_batch, tokenizer)
  return prediction

In [360]:
prompt= [
    [system_prompt, post_messages]
]
tokenized = tokenizer.apply_chat_template(prompt, padding= False, return_tensors ="pt", add_special_tokens= False).to(device)
op_batch = model.generate(tokenized, max_new_tokens = 20)
decode = tokenizer.batch_decode(op_batch)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [361]:
decode

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 24 Jan 2026\n\nYou are an AI system that reads an Title and Book Description and classifies category of the book applied, you must\n    choose from the following classes:\n    Labeled Category:Sequential Art\n or Labeled Category:Classics\n or Labeled Category:Humor\n or Labeled Category:Biography\n or Labeled Category:Horror\n or Labeled Category:Young Adult\n or Labeled Category:Romance\n or Labeled Category:Autobiography\n or Labeled Category:Fiction\n or Labeled Category:Business\n or Labeled Category:Default\n or Labeled Category:Psychology\n or Labeled Category:Nonfiction\n or Labeled Category:New Adult\n or Labeled Category:Mystery\n or Labeled Category:Poetry\n or Labeled Category:Historical Fiction\n or Labeled Category:History\n or Labeled Category:Fantasy\n or Labeled Category:Add a comment\n or Labeled Category:Food and Drink\n or Labeled Category:Thriller\n o

In [364]:
  labels  = [d.split("<|start_header_id|>assistant<|end_header_id|>\n\nLabeled Category:")[0].split("<|eot_id|>")[0].strip() for d in decode_batch]
  labels

["{'role':'system', 'content':'You are an AI system that reads an Title and Book Description and classifies category of the book applied, you must\\n    choose from the following classes:\\n    Labeled Category:Sequential Art\\n or Labeled Category:Classics\\n or Labeled Category:Humor\\n or Labeled Category:Biography\\n or Labeled Category:Horror\\n or Labeled Category:Young Adult\\n or Labeled Category:Romance\\n or Labeled Category:Autobiography\\n or Labeled Category:Fiction\\n or Labeled Category:Business\\n or Labeled Category:Default\\n or Labeled Category:Psychology\\n or Labeled Category:Nonfiction\\n or Labeled Category:New Adult\\n or Labeled Category:Mystery\\n or Labeled Category:Poetry\\n or Labeled Category:Historical Fiction\\n or Labeled Category:History\\n or Labeled Category:Fantasy\\n or Labeled Category:Add a comment\\n or Labeled Category:Food and Drink\\n or Labeled Category:Thriller\\n or Labeled Category:Religion\\n or Labeled Category:Womens Fiction\\n or Labe

In [324]:
prompt = [
    [str(system_prompt), str(post_messages)]
  ]

In [358]:
tokenized = tokenizer(prompt, padding= False, return_tensors ="pt", add_special_tokens= False).to(device)
output_batch = model.generate(input_ids = tokenized["input_ids"], attention_mask =tokenized["attention_mask"], max_new_tokens = 20, do_sample= False, temperature = 0, top_p =1)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [359]:
decode_batch = tokenizer.batch_decode(output_batch)
decode_batch

["{'role':'system', 'content':'You are an AI system that reads an Title and Book Description and classifies category of the book applied, you must\\n    choose from the following classes:\\n    Labeled Category:Sequential Art\\n or Labeled Category:Classics\\n or Labeled Category:Humor\\n or Labeled Category:Biography\\n or Labeled Category:Horror\\n or Labeled Category:Young Adult\\n or Labeled Category:Romance\\n or Labeled Category:Autobiography\\n or Labeled Category:Fiction\\n or Labeled Category:Business\\n or Labeled Category:Default\\n or Labeled Category:Psychology\\n or Labeled Category:Nonfiction\\n or Labeled Category:New Adult\\n or Labeled Category:Mystery\\n or Labeled Category:Poetry\\n or Labeled Category:Historical Fiction\\n or Labeled Category:History\\n or Labeled Category:Fantasy\\n or Labeled Category:Add a comment\\n or Labeled Category:Food and Drink\\n or Labeled Category:Thriller\\n or Labeled Category:Religion\\n or Labeled Category:Womens Fiction\\n or Labe

In [350]:
def extract_labels_from_output(decode_batch, tokenizer):
  # labels = []
  # for d in decode_batch:
  #   if "Labeled Category:" in d:
  #       label = d.split("Labeled Category:")[1].split("<|eot_id|>")[0].strip()
  #       return labels
  #   else:
  #       label = None
  #   labels.append(label)
  deo
  labels = [d.split("<|start_header_id|>assistant<|end_header_id|>\n\nLabeled Category:")[0].split("<|eot_id|>")[0].strip() for d in decode_batch]
  print(labels)
  # return labels

In [354]:

 labels  = [d.split("<|start_header_id|>assistant<|end_header_id|>\n\nLabeled Category:")[0].split("<|eot_id|>")[0].strip() for d in decode_batch]
  print(labels)

["{'role':'system', 'content':'You are an AI system that reads an Title and Book Description and classifies category of the book applied, you must\\n    choose from the following classes:\\n    Labeled Category:Sequential Art\\n or Labeled Category:Classics\\n or Labeled Category:Humor\\n or Labeled Category:Biography\\n or Labeled Category:Horror\\n or Labeled Category:Young Adult\\n or Labeled Category:Romance\\n or Labeled Category:Autobiography\\n or Labeled Category:Fiction\\n or Labeled Category:Business\\n or Labeled Category:Default\\n or Labeled Category:Psychology\\n or Labeled Category:Nonfiction\\n or Labeled Category:New Adult\\n or Labeled Category:Mystery\\n or Labeled Category:Poetry\\n or Labeled Category:Historical Fiction\\n or Labeled Category:History\\n or Labeled Category:Fantasy\\n or Labeled Category:Add a comment\\n or Labeled Category:Food and Drink\\n or Labeled Category:Thriller\\n or Labeled Category:Religion\\n or Labeled Category:Womens Fiction\\n or Labe

In [333]:
def test_model(dataloader, model, tokenizer):
  comparison_df= {
      "predictions": [],
      "labels": []
  }

  for batch in dataloader:
    predictions = generate_outputs(prompts= prompt, model = model, tokenizer= tokenizer) # prompts = batch["prompt"]
    comparison_df["labels"].extend(batch["Category"])
    comparison_df["predictions"].extend(predictions)

  # comparison_df = pd.DataFrame(comparison_df)
  # accuracy = (comparison_df["labels"] == comparison_df["predictions"]).mean()
  # num_invalid_pred = (~comparison_df["predictions"].isin(valid_category)).mean()
  # print(comparison_df.head(10))
  # return {"accuracy ": accuracy,"invalid_predictions": num_invalid_pred}
  return comparison_df

In [334]:
from datasets import Dataset #defined by huggingface not pandas or tf
num_test_example= 1
batch_size = 16
# _, test_df = get_dataset(tokenizer = tokenizer, train_size = 0, test_size = num_test_example)
test_Dataset = Dataset.from_pandas(df_test)
test_dataloader = DataLoader(test_Dataset, batch_size = batch_size, shuffle= False)
metrics = test_model(test_dataloader, model, tokenizer)
# print("\n".join([f"{k} = {v}" for k, v in metrics.items()]))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [336]:
print(metrics["predictions"])

[]


In [340]:
print(len(metrics["labels"]))

200


dataloader is object but we need list

In [370]:
for i in test_dataloader:
  print(i["Title"], i["Book_Description"])

['Les Fleurs du Mal', 'Adultery', 'The Story of Art', 'Ash', 'Eleanor & Park', 'How to Stop Worrying and Start Living', 'Luckiest Girl Alive', 'Logan Kade (Fallen Crest High #5.5)', 'Imperfect Harmony', 'Someone Like You (The Harrisons #2)', "The White Queen (The Cousins' War #1)", 'Vogue Colors A to Z: A Fashion Coloring Book', "Brazen: The Courage to Find the You That's Been Hiding", 'Twenties Girl', 'The Loney', 'Poses for Artists Volume 1 - Dynamic and Sitting Poses: An Essential Reference for Figure Drawing and the Human Form'] ["Presents the first American translation of the complete text of Baudelaire's 1857 masterwork and includes the complete original French texts for easy comparison.", "I want to change. I need to change. I'm gradually losing touch with myself. Adultery, the provocative new novel by Paulo Coelho, best-selling author of The Alchemist and Eleven Minutes, explores the question of what it means to live life fully and happily, finding the balance between life's ro